**AKZOUN, Hafsa, 21511721.**

**Rayane CHERIFI, 21510913**.

**Which LLM(s) did you use for this work?**

# https://github.com/balouf/progres 

# PROGRES - TME2

Fabien Mathieu - fabien.mathieu@lip6.fr

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

**Note**: Star exercises (indicated by *) should only be done if all other exercises have been completed. You 
don't have to do them if you do not want.

# Rules

1. Cite your sources
2. One file to rule them all
3. Explain
4. Execute your code


https://github.com/balouf/progres/blob/main/rules.ipynb

The rules always apply unless an explicit exception is made.

# About this TME

The exercises require skills and knowledge not seen in course. The goal is to learn how to use LLM(s) to make them.

You grade will not be based on the actual completion of the exercises, but on how you solve them.

# Exercise 1 - Pass a technical job interview

You passed the HR interview with Swapcard. Now is the time to perform the live coding exercise.

Answer the questions from https://gist.github.com/balouf/ee28114beeb1545aed7b682f0af36bd9

Note: this is a real exercise I used last year to hire a new developer. Unlike you, applicants were not allowed to use LLMs.

You answer here (code/markdown, include your prompts)

Prompt utilisé:

I’m preparing for a live coding interview with Swapcard. The exercise involves two compressed text files: 

- *ingredients.txt.gz* which is a list of ingredients grouped by category 

This file lists common ingredients grouped by category. Each category (Proteins, Vegetables, Fruits,etc.) is followed by a list of corresponding ingredients separated by commas

- *recipes.txt.gz* (a list of recipes with their ingredients). 

This file contains recipes listed line by line. Each line starts with a recipe identifier (e.g., `Recipe #1:`) followed by a comma-separated list of ingredients used in that recipe.

The goal is to determine how many recipes do not contain any ingredients from the categories “Proteins” or “Dairy.” 

Please write an efficient and well-commented Python script that reads the .gz files using the gzip library, dynamically extracts the categories and ingredients, and counts the recipes that exclude those categories. The code should handle large files efficiently and include a concise explanation of its operation

For more context, I will be using Jupiter Notebook 

Fisrt let focus on solving the first question

In [ ]:
import gzip

# Load ingredients and categorize them
def load_ingredients(file_path):
    categories = {}
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if ':' in line:
                category, items = line.split(':', 1)
                categories[category.strip()] = set(i.strip() for i in items.split(','))
    return categories

# Load recipes
def load_recipes(file_path):
    recipes = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Extract ingredients
            if ':' in line:
                _, ingredients_str = line.split(':', 1)
                ingredients = set(i.strip() for i in ingredients_str.split(','))
                recipes.append((line, ingredients))
    return recipes

# Main
ingredients_file = 'ingredients.txt.gz'
recipes_file = 'recipes.txt.gz'

categories = load_ingredients(ingredients_file)
proteins_dairy = categories.get('Proteins', set()) | categories.get('Dairy', set())

recipes = load_recipes(recipes_file)

# Count recipes without Proteins or Dairy
count_no_protein_dairy = sum(1 for _, ing in recipes if ing.isdisjoint(proteins_dairy))
print(f"Recipes do not contain any ingredient from category Proteins or Dairy: {count_no_protein_dairy}")


recipes do not contain any ingredient from category Proteins or Dairy: 3432


In [22]:
# Find recipes with all types of ingredients
recipes_with_all_categories = []

for line, ingredients in recipes:
    if all(ingredients & cat_ingredients for cat_ingredients in categories.values()):
        recipes_with_all_categories.append(line)

# Print the recipes
for r in recipes_with_all_categories:
    print(r)

Recipe #3763: corn, tofu, apple cider vinegar, parsley, raspberry, pasta, hot sauce, vanilla extract, butter


In [23]:
import random

# Step 1: Assign mock weights to all ingredients
all_ingredients = set()
for ing_set in categories.values():
    all_ingredients.update(ing_set)

# Assign a random weight between 1 and 10 to each ingredient
ingredient_weights = {ing: random.randint(1, 10) for ing in all_ingredients}

# Step 2: Calculate recipe scores
recipe_scores = []
for line, ingredients in recipes:
    score = sum(ingredient_weights.get(ing, 0) for ing in ingredients)
    recipe_scores.append((score, line))

# Step 3: Find the recipe with the highest score
favorite_recipe = max(recipe_scores, key=lambda x: x[0])

print("Favorite recipe based on mock weights:")
print(favorite_recipe[1])
print("Score:", favorite_recipe[0])

Favorite recipe based on mock weights:
Recipe #9632: olive oil, chicken, white wine vinegar, barbecue sauce, banana, spinach, lentils, cinnamon, sour cream
Score: 79


# Exercise 2 - Produce scientific code

You are hired. Now is the time to show that you can produce some code.

- Write a Python function that computes the Pareto front of a set of 2D points (bigger values are better).

What is the complexity of the function?

Benchmark your function: write a function that uses as input and that returns the time to compute the Pareto front of randomly generated points.

Display the actual performance and compare it to the theoretical performance.


(optional) Try do improve the speed of your code*.

(optional) Do the same thing for 3D points*.

(optional) Do the same thing for 4D points*.

# Exercise 3 - Decypher

You decide to learn about cybersecurity. Breaking cyphers seems a nice entry points. Choose your language and solve the problem. You are expected to code the decyphering function, but the design of that function can be "manual".

## French version (from *Énigmes du journal Le Monde*)

La version française est en deux étapes (simple, compliqué)

Un chiffrement par substitution est un type de code secret dans lequel chaque lettre est remplacée par une autre. Il est par exemple possible de simplement décaler chaque lettre d’un certain nombre de rangs dans l’alphabet.

In [1]:
txt = """
Oh soxv fhoheuh gh fhv frghv hvw vdqv grxwh oh fkliiuh gh Fhvdu, 
xwlolvh sdu Mxohv gdqv vhv fruuhvsrqgdqfhv frqilghqwlhoohv. 
Lo frqvlvwh d ghfdohu fkdtxh ohwwuh gh wurlv udqjv, 
pdlv mh qh yrxv dssuhqgv ulhq sxlvtxh yrxv hwhv hq wudlq gh oluh fhwwh skudvh.
"""

You answer here (code/markdown, include your prompts)

In [3]:
caesar(txt)[:15]

'Le plus celebre'

Si l’on se contente d’un tel décalage, il y a autant de possibilités que de lettres, c’est-à-dire 26 (ou 25 si on exclut le décalage de 0 rang qui laisse le message inchangé). Mais si l’on mélange complètement les lettres, le nombre de combinaisons explose : il y a 403 291 461 126 605 635 584 000 000 (403 quadrillions et des poussières) permutations des vingt-six lettres, ce qui rend impossible de toutes les tester, même informatiquement. Est-il pour autant impossible de déchiffrer un tel code ?

In [4]:
txt = """O qhxnhx, xs c o ew vhet qsevnv qor los, 
qneh fex jneyhoxv ojnxh so rnsevxnw y’ew tpohobxo o rebrvxvevxnw : 
so tnlqohoxrnw rvovxrvxz [rxt]. 
Xs c o qser y’O fe’xs w’c o y’X yowr ew yxrtnehr uhowtoxr. 
Oerrx, xs c o qser y’X fe’xs w’c o y’E ne y’E fe’xs w’c o y’C… 
Xs reuuxv ynwt, qohoxv-xs, y’ottnlqsxh so tnlqohoxrnw qneh ojnxh so rnsevxnw. 
Roeu rx s’xwyxjxye [lnx, L. Soewoc] fex o qnwye s’nhxgxwos tpohobxo, 
tnwvnehwowv so snx, w’ero doloxr yowr ro unhlesovxnw ye qser bowos : 
sex, so, fex o ro qnrxvxnw rexjowv Y ne ojowv U. 
Rx ve or tnlqhxr to : bhojn !"""

In [6]:
decypher(txt, dic)[0]

'a'

You answer here (code/markdown, include your prompts)

## English version (classical cypher from Edgar Poe, XIXth century)

Decypher the following message. Each character corresponds to one letter.

In [7]:
txt = """5 3‡‡+ 305)) 6* ;48 26)4‡. ) 4‡);80 6* ;48 +8¶60 ) )85; 1‡(;: ‡*8 
+83(88) 5*+ ;46(;88* 96*?;8) *‡(;4 85); 5*+ 2: *‡(;4 956* 2(5*—4 
)8¶8*;4 0692 85); )6+8 )4‡‡; 1(‡9 ;48 081; 8:8 ‡1 ;48 +85;4 ) 485+ 
5 288 06*8 1(‡9 ;48 ;(88 ;4(‡?34 ;48 )4‡; 161;: 188; ‡?;"""

In [9]:
decypher(txt, dic)[0]

'a'

# Exercise 4 - Jailbreaking a LLM

To improve your skills in cybersecurity, you decide to learn LLM jailbreak techniques.

Go to https://gandalf.lakera.ai/baseline and pass levels up to level 6 included.

Optional*: pass levels 7 and 8.

**NOTE:** for this exercise, no code required. Just give the prompt you used and explain how you came to find it. Remind: if you googled the way to solve this exercise, you must tell that and indicate your sources. For example:

**Level 1:** the site tells I just need to ask, so this is exactly what I did:

> The password was: 

You answer here (markdown, include your prompts)